<h1 style = "text-align:center;color:blue;font-family:algerian;font-size:35px;">Classifiez des biens de consommation</h1>

<h1 style = "text-align:center;color:blue;font-size:25px;">Bassirou_Chaka_Moussa</h1>
<h1 style = "text-align:center;color:red;font-size:25px;">élèves ingénieurs statisticiens économistes</h1>

## **Summary**

Cette étude examine la **faisabilité** de concevoir un système automatisé de classification d’articles en différentes catégories, en se basant notamment sur des **images illustratives** ou du **texte en anglais**. L’objectif est d’atteindre une précision suffisante pour garantir une attribution des catégories à la fois fiable et autonome. L’analyse repose sur un jeu de données comprenant **15 variables** et **1 050 observations**.

<div style="background-color:#d4f0ff; font-size:20px;padding:20px; border-left:25px solid #1c8adb;">
    <strong>Packages </strong> 
</div>Dans cette section, nous importons les bibliothèques nécessaires.

In [1]:
%load_ext autoreload
%autoreload 2

In [7]:
import warnings
warnings.filterwarnings("ignore")

import math
import pandas as pd
import numpy as np
import re
from unidecode import unidecode
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.image import imread
import time
import random
import operator
import scipy.cluster.hierarchy as shc
import multiprocessing
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize, FreqDist
from sklearn import (preprocessing,
                     manifold,
                     decomposition)
from sklearn.cluster import AgglomerativeClustering, KMeans, DBSCAN
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.metrics import pairwise_distances_argmin_min
from sklearn.metrics import silhouette_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.decomposition import NMF
from PIL import Image, ImageOps, ImageFilter
#import tensorflow as tf
#import keras
#from keras.preprocessing import image
import cv2 as cv

<div style="background-color:#d4f0ff; font-size:20px;padding:20px; border-left:25px solid #1c8adb;">
    <strong> Importation des fonctions utiles </strong> 
</div>Afin d'améliorer la lisibilité et d'assurer un meilleur confort de lecture de ce notebook, les fonctions dédiées à la manipulation des données et à la création des graphiques ont été centralisées dans le fichier project_functions.py.


Ces fonctions sont ensuite importées grâce à la ligne de code suivante :

In [136]:
import project_functions as pf

<div style="background-color:#d4f0ff; font-size:20px;padding:20px; border-left:25px solid #1c8adb;">
    <strong> Data collection </strong> 
</div> Dans cette partie nous allons importer le jeu des données

In [9]:
df = pd.read_csv("C:/Users/hp/Desktop/Projet_ML/Base/flipkart_com-ecommerce_sample_1050(2).csv")

<div style="background-color:#d4f0ff; font-size:20px;padding:20px; border-left:25px solid #1c8adb;">
    <strong> 1. DESCRIPTION DU JEU DE DONNEES </strong> 
</div>

___
### **1.1 Apperçu des données**

In [10]:
df.head()

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
0,55b85ea15a1536d46b7190ad6fff8ce7,2016-04-30 03:22:56 +0000,http://www.flipkart.com/elegance-polyester-mul...,Elegance Polyester Multicolor Abstract Eyelet ...,"[""Home Furnishing >> Curtains & Accessories >>...",CRNEG7BKMFFYHQ8Z,1899.0,899.0,55b85ea15a1536d46b7190ad6fff8ce7.jpg,False,Key Features of Elegance Polyester Multicolor ...,No rating available,No rating available,Elegance,"{""product_specification""=>[{""key""=>""Brand"", ""v..."
1,7b72c92c2f6c40268628ec5f14c6d590,2016-04-30 03:22:56 +0000,http://www.flipkart.com/sathiyas-cotton-bath-t...,Sathiyas Cotton Bath Towel,"[""Baby Care >> Baby Bath & Skin >> Baby Bath T...",BTWEGFZHGBXPHZUH,600.0,449.0,7b72c92c2f6c40268628ec5f14c6d590.jpg,False,Specifications of Sathiyas Cotton Bath Towel (...,No rating available,No rating available,Sathiyas,"{""product_specification""=>[{""key""=>""Machine Wa..."
2,64d5d4a258243731dc7bbb1eef49ad74,2016-04-30 03:22:56 +0000,http://www.flipkart.com/eurospa-cotton-terry-f...,Eurospa Cotton Terry Face Towel Set,"[""Baby Care >> Baby Bath & Skin >> Baby Bath T...",BTWEG6SHXTDB2A2Y,NaN,NaN,64d5d4a258243731dc7bbb1eef49ad74.jpg,False,Key Features of Eurospa Cotton Terry Face Towe...,No rating available,No rating available,Eurospa,"{""product_specification""=>[{""key""=>""Material"",..."
3,d4684dcdc759dd9cdf41504698d737d8,2016-06-20 08:49:52 +0000,http://www.flipkart.com/santosh-royal-fashion-...,SANTOSH ROYAL FASHION Cotton Printed King size...,"[""Home Furnishing >> Bed Linen >> Bedsheets >>...",BDSEJT9UQWHDUBH4,2699.0,1299.0,d4684dcdc759dd9cdf41504698d737d8.jpg,False,Key Features of SANTOSH ROYAL FASHION Cotton P...,No rating available,No rating available,SANTOSH ROYAL FASHION,"{""product_specification""=>[{""key""=>""Brand"", ""v..."
4,6325b6870c54cd47be6ebfbffa620ec7,2016-06-20 08:49:52 +0000,http://www.flipkart.com/jaipur-print-cotton-fl...,Jaipur Print Cotton Floral King sized Double B...,"[""Home Furnishing >> Bed Linen >> Bedsheets >>...",BDSEJTHNGWVGWWQU,2599.0,698.0,6325b6870c54cd47be6ebfbffa620ec7.jpg,False,Key Features of Jaipur Print Cotton Floral Kin...,No rating available,No rating available,Jaipur Print,"{""product_specification""=>[{""key""=>""Machine Wa..."


Chaque ligne représente un produit, incluant son nom ainsi que ses caractéristiques associées telles que la description, la catégorie, la marque, une image, etc.

___
### **1.2 Structure des données**

In [ ]:
df.info()

In [ ]:
pf.describe_dataset({'sample e-commerce':[df, "Produits"]})

Le jeu de données contient 1050 lignes et 15 colonnes.

___
### **1.3 Données manquantes**

In [ ]:
pf.plot_percentage_missing_values_for(df, 20, 20)

De manière générale, les colonnes sont très bien renseignées (avec un taux de remplissage supérieur à 90 %), à l’exception de la colonne "brand", qui présente plus de 30 % de valeurs manquantes.Alors nous allons remplacer les valeurs manquantes de brand par " ".

Pour un projet de classification, remplacer les valeurs manquantes de brand par une chaîne vide (" ") conserve la nature textuelle de la variable et facilite son encodage sans introduire de biais. Cette méthode est bien tolérée par les algorithmes comme les arbres ou forêts aléatoires, qui gèrent efficacement les modalités neutres, assurant ainsi un prétraitement simple et adapté au pipeline de classification.

In [17]:
# Les valeurs manquantes pour la variable "brand" sont remplacés par "".
df["brand"].fillna("", inplace=True)

___
### **1.4 Les catégories produits**

Chaque produit est classé selon une hiérarchie à plusieurs niveaux, représentée sous forme d’arborescence. Les informations relatives à cette classification sont contenues dans la colonne **product_category_tree.**

In [ ]:
# Exemple de la catégorie d'un produit
print(df["product_category_tree"][0])

In [ ]:
# On récupère la colomne "product_category_tree"
list_categories_0, list_categories_1 = [], []
for txt in df["product_category_tree"] :
    
    list_categories_0.append(txt.split(">>")[0].split("\"")[1].strip()) # split du nom par >> et on récupère la première partie
    list_categories_1.append(txt.split(">>")[1].strip()) # split du nom par >> et on récupère la première partie

# Création d'une nouvelle série catégorie    
df["category_0"] = pd.Series(list_categories_0)
df["category_1"] = pd.Series(list_categories_1)

print("Il y a {} catégories de niveau 0 et {} categories de niveau 1".format(df["category_0"].nunique(), df["category_1"].nunique()))

#### Affichage des catégories de niveau 0

In [ ]:
font_title = {"weight": "bold",
              "size": 30}


# Liste de couleurs correspondant à chaque catégorie (dans l'ordre de df["category_0"].value_counts().index)
colors = ["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd", "#8c564b"]  # adapte la taille selon le nombre de catégories

fig = plt.figure(figsize=(10, 6))
sns.countplot(
    data=df, 
    y="category_0",
    order=df["category_0"].value_counts().index,
    palette=colors  # applique la liste de couleurs
)
plt.yticks(size=14)
plt.title(f"Nombre de produits par catégorie de niveau 0\n", fontdict=font_title)
plt.ylabel("product_category level 0", fontsize=18)
plt.show()


#### Affichage des catégories de niveau 1

In [ ]:
fig = plt.figure(figsize=(18, 8))
sns.countplot(data=df, x="category_1",
              edgecolor="black",
              order = df['category_1'].value_counts().index,
              alpha=0.7)
plt.xticks(rotation=90, size=14)
plt.title(f"Nombre de produits par catégorie de niveau 1\n",
          fontdict=font_title)
plt.xlabel('\n product_category level 1', fontsize=26)
plt.show()

In [ ]:
# Récupérer l'ordre des catégories
order = df['category_1'].value_counts().index

# Créer une liste de couleurs (à adapter selon le nombre de catégories)
colors = sns.color_palette("hsv", len(order))  # palette automatique avec autant de couleurs que de catégories

fig = plt.figure(figsize=(18, 8))
sns.countplot(
    data=df,
    x="category_1",
    edgecolor="black",
    order=order,
    alpha=0.7,
    palette=colors  # appliquer les couleurs spécifiques
)
plt.xticks(rotation=90, size=14)
plt.title(f"Nombre de produits par catégorie de niveau 1\n", fontdict=font_title)
plt.xlabel('\n product_category level 1', fontsize=26)
plt.show()

In [28]:
# Les catégories utilisés pour la résolution du problème de segmentation
categories = df["category_0"]

___
### **1.5 Descriptif des produits**

Nous allons donne un exemple de texte descriptif d'un produit

In [ ]:
des = df.loc[df.uniq_id == "3c4ca34c50a5437a1bcc42b72fc1351f"]

print("La catégorie du produit est : {} \n".format(des["category_0"].iloc[0]))

print("La marque du produit est : {} \n".format(des["brand"].iloc[0]))

print("Le nom du produit est : {} \n".format(des["product_name"].iloc[0]))

print("Le descriptif du produit est : \n \n {}".format(des["description"].iloc[0]))

___
### **1.6 Vérification des doublons**

In [ ]:
format(len(df)!=df["uniq_id"].nunique())

Le résultat est **False**, c'est à dire que notre dataset ne contient pas des doublons

___
### **1.7 Concaténation des variables textes : "product_name", "description" et "brand"**

In [ ]:
products_text_data = pd.DataFrame()
products_text_data["uniq_id"] = df["uniq_id"]
products_text_data["category"] = df["category_0"]
products_text_data["description"] = df["product_name"] + " "+ df["brand"] + " " + df["description"]
products_text_data

___
### **1.8 Les images**

#### Affichage d'exemples d'images par catégorie produit de niveau 0

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.image import imread
import os

path = r"C:\Users\hp\Desktop\Projet_ML\Base\Images"

for j, categ in enumerate(categories.unique()):
    print("\033[1m" + categ + "\033[0m")
    plt.figure(figsize=(12, 4))

    for i in range(3):
        name_image = df[categories == categ]["image"].iloc[i]
        filename = os.path.join(path, name_image)

        if not os.path.exists(filename):
            print(f"❌ Fichier introuvable : {filename}")
            continue

        img = imread(filename)
        plt.subplot(1, 3, i + 1)
        plt.imshow(img)
        plt.axis('off')
        plt.title(f"Exemple {i+1}", fontsize=10)

    plt.suptitle(f"Catégorie : {categ}", fontsize=14, weight='bold')
    plt.tight_layout()
    plt.show()


<div style="background-color:#d4f0ff; font-size:20px;padding:20px; border-left:25px solid #1c8adb;">
    <strong> 2. NETTOYAGE ET PRÉ-TRAITEMENT </strong> 
</div>

### **2.1 Textes**

- Suppression des ponctuations
- Tokenization
- Suppression des tokens de taille < 2
- Suppression des stopwords
- Stemming
- Lemmatization
- Conversion des mots en vecteurs de features (BOW, TF-IDF)

### **2.1.1 Suppression des ponctuations**

Dans cette étape, nous supprimons tous les signes de ponctuation (tels que ., !, ?, ;, :) présents dans le texte. Cette opération vise à épurer les données textuelles en éliminant les caractères qui n’ont pas de valeur sémantique directe, ce qui simplifie l’analyse et le traitement ultérieur des mots.

In [ ]:
products_text_data['removed_punc'] = products_text_data['description'].apply(lambda x: pf.remove_punct(x))
products_text_data.head(3)

#### **2.1.2 Tokenization**

Dans cette étape, le texte est segmenté en unités élémentaires appelées tokens, qui correspondent le plus souvent à des mots, mais peuvent également être des phrases ou des caractères selon le niveau de granularité souhaité.

In [ ]:
products_text_data['tokens'] = products_text_data['removed_punc'].apply(lambda msg : pf.tokenize(msg))
products_text_data.head(3)

#### **2.1.3 Suppression des tokens de taille < 2**

In [ ]:
products_text_data['larger_tokens'] = products_text_data['tokens'].apply(lambda x : pf.remove_small_words(x))
products_text_data.head(3)

#### **2.1.4 Suppression des stopwords**

In [ ]:
products_text_data['clean_tokens'] = products_text_data['larger_tokens'].apply(lambda x : pf.remove_stopwords(x))
products_text_data.head(3)

#### **2.1.5 Stemming**

In [ ]:
products_text_data['stem_words'] = products_text_data['clean_tokens'].apply(lambda wrd: pf.stemming(wrd))
products_text_data.head(3)

#### **2.1.6 Lemmatization**

In [ ]:
products_text_data['lemma_words'] = products_text_data['clean_tokens'].apply(lambda x : pf.lemmatize(x))
products_text_data.head(3)

In [ ]:
products_text_data['clean_text'] = products_text_data['lemma_words'].apply(lambda x : pf.return_sentences(x))
products_text_data.head(3)

#### Texte illustratif avant et après prétraitement

In [ ]:
des = products_text_data.loc[products_text_data.uniq_id == "3c4ca34c50a5437a1bcc42b72fc1351f"]
print("Le descriptif avant prétraitement est : \n \n {}".format(des["description"].iloc[0])) 

In [ ]:
print("Le descriptif après prétraitement est : \n \n {}".format(des["clean_text"].iloc[0])) 

#### Affichage des Top 10 mots les plus fréquents pour chaque catégorie produit de niveau 0

In [ ]:
most_freq_dfs = pd.DataFrame()

for cat1, data_df in products_text_data.groupby("category"):
    most_freq_df = pf.get_most_freq(data_df["lemma_words"], 10).reset_index()
    most_freq_df["Category"] = cat1
    most_freq_dfs = pd.concat([most_freq_dfs, most_freq_df])
    pf.plot_freq_dist(most_freq_df, cat1 + " - FRÉQUENCE DES MOTS (Top 10)", 10, 5)

#### Wordclouds par categories produit

In [ ]:
gb = products_text_data.groupby('category')['clean_text']

pf.plot_wordclouds_from_gb(gb, n_top_words=30, n_rows=4, figsize=(15,10),
                        backgnd_color='beige', cmap='tab10', random_state=14)

#### **2.1.7 Conversion des mots en vecteurs de features**

Nous allons extraire les features en utilisant les méthodes suivantes:

- Bag of Words (BOW)
- Term Frequency - Inverse Document Frequency (TF-IDF)
- Word2Vec

Nous allons comparer les résultats obtenus avec chacune de ces transformations.

##### **2.1.7.1 Transformation en matrice BOW**

In [ ]:
bow = CountVectorizer()
bow_vect = bow.fit_transform(products_text_data['clean_text']).toarray()
bow_vect.shape

##### **2.1.7.2 Transformation en matrice TF-IDF**

In [ ]:
tfidf = TfidfVectorizer()
tfidf_vect = tfidf.fit_transform(products_text_data['clean_text']).toarray()
tfidf_vect.shape

##### **2.1.7.3 Transformation en matrice Word2Vec**

In [ ]:
from gensim.models import Word2Vec

data_vec =products_text_data['clean_text']
data_token = products_text_data['lemma_words']
cores = multiprocessing.cpu_count()
text_dim=100

In [ ]:
# Instanciation du modèle
w2v_model_desc_lem = Word2Vec(min_count=20, window=3, size=text_dim,
                              sample=6e-5, alpha=0.03, min_alpha=0.0007,
                              negative=20, workers=cores-1)
# Construit le vocabulaire à partir d'une séquence de mots et
# initialise ainsi le modèle.
w2v_model_desc_lem.build_vocab(data_token,
                               progress_per=10000)
# Vectorisation
vector_w2v_desc_lem = pf.word2vec_vectorisation(data_vec,
                                                        text_dim,
                                                        w2v_model_desc_lem)

In [ ]:
vector_w2v_desc_lem.shape

### 2.2 Images

- **PIL** (Pyhton Imaging Library) et **OpenCV** (Open Compute Vision) sont des librairies qui comprennent un ensemble de méthodes permettant de charger, traiter les images (redimensionnement, filtres...). Ces librairies seront utilisées pour le pré-traitement des images.

- Le traitement sera effectué en 5 étapes :
    - Correction de l'exposition (étirement d'histogramme) 
    - Correction du contraste (égalisation d'histogramme)    
    - Réduction du bruit (filtre)
    - Conversion en niveau de gris     
    - Redimensionnement (en 224 * 224)


In [93]:
# Dataframe de travail pour le traitement des images

products_image_data = pd.DataFrame()
products_image_data["image"] = df["image"]
products_image_data['image_loc'] = [path + row for row in df['image']]
products_image_data["category"] = df["category_0"]

products_image_data.head(3)

,image,image_loc,category
0,55b85ea15a1536d46b7190ad6fff8ce7.jpg,C:\Users\hp\Desktop\Projet_ML\Base\Images55b85...,Home Furnishing
1,7b72c92c2f6c40268628ec5f14c6d590.jpg,C:\Users\hp\Desktop\Projet_ML\Base\Images7b72c...,Baby Care
2,64d5d4a258243731dc7bbb1eef49ad74.jpg,C:\Users\hp\Desktop\Projet_ML\Base\Images64d5d...,Baby Care


___
#### **2.1 Illustration du pré-traitement sur une image**

In [94]:
dim = (224, 224)

## ***Chargement de l'image originale***

In [ ]:
# Chargement de l'image originale comme matrice de pixels
img_orig = Image.open(r"C:\Users\hp\Desktop\Projet_ML\Base\Images\3c4ca34c50a5437a1bcc42b72fc1351f.jpg")
pf.afficher_image_histopixel(img_orig, 'Image originale')

## ***Correction de l'exposition (étirement d'histogramme)***

- Image sous-exposée ==> histogramme concentré à droite vers les niveaux de gris faible (255 = blanc).
- Image sur-exposée ==> histogramme concentré à gauche vers les niveaux de gris fort (0 = noir).

In [ ]:
# Correction de l'exposition 
img_expo = ImageOps.autocontrast(img_orig, 1)
pf.afficher_image_histopixel(img_expo,'Image après correction de l\'exposition')

## ***Correction du contraste (égalisation d'histogramme)***

Le contraste caractérise la répartition de lumière dans une image : plus une image est contrastée, plus la différence de luminosité entre ses zones claires et sombres est importante. En général, une image peu contrastée est terne, tandis qu'une image trop contrastée est visuellement "agressive". Dans les deux cas, l'image manque de clarté car certains de ses détails seront peu, voire pas du tout,  visibles.

In [ ]:
# Correction du contraste
img_contr_pils = ImageOps.equalize(img_expo)
pf.afficher_image_histopixel(img_contr_pils,'Image après correction du contraste')

## ***Réduction du bruit (lissage)***

In [ ]:
# Réduction du bruit (filtre)

img_filter = img_contr_pils.filter(ImageFilter.BoxBlur(1))
pf.afficher_image_histopixel(img_filter,'Image après réduction du bruit')

## ***Convertir l'image en niveau de gris***

In [ ]:
# Conversion de l'image en niveau de gris pour SIFT
img_gris = cv.cvtColor(np.array(img_filter), cv.COLOR_RGB2GRAY)
pf.afficher_image_histopixel(img_gris,'Image en gris')

## ***Redimensionnement en 224 * 224***

In [ ]:
# Redimensionnement image contraste PILS
img_redim_pils = cv.resize(np.array(img_gris), dim,
                            interpolation=cv.INTER_AREA)
pf.afficher_image_histopixel(img_redim_pils,'Image redimensionnée 224*224')

___
#### **2.2 Pré-traitement de toutes les images**

In [ ]:
# Pré-traitement de toutes les images
products_image_data['image_proces'] = r"C:\Users\hp\Desktop\Projet_ML\Base\Images"
products_image_data['image_loc'].apply(pf.preprocess_image)

In [117]:
#print(products_image_data["image_loc"].head())

In [113]:
# Vérifier quels fichiers sont réellement absents
import os

# Ajout d'une colonne indiquant l'existence de l'image
products_image_data["image_exists"] = products_image_data["image_loc"].apply(os.path.isfile)

# Séparer les lignes avec et sans image
images_trouvees = products_image_data[products_image_data["image_exists"] == True]
images_manquantes = products_image_data[products_image_data["image_exists"] == False]

# Sauvegarde facultative des lignes manquantes
images_manquantes.to_csv("images_manquantes.csv", index=False)
print(f"[✅] {len(images_manquantes)} images manquantes ont été enregistrées dans 'images_manquantes.csv'")


[✅] 1050 images manquantes ont été enregistrées dans 'images_manquantes.csv'


In [148]:
def safe_preprocess(image_path):
    try:
        return pf.preprocess_image(image_path)
    except Exception as e:
        print(f"[⚠️] Erreur pour {image_path} : {e}")
        return None

# Appliquer uniquement aux images disponibles
images_trouvees["image_proces"] = images_trouvees["image_loc"].apply(safe_preprocess)


In [149]:
products_image_data = products_image_data.drop(columns=["image_proces"], errors="ignore")
products_image_data = products_image_data.merge(
    images_trouvees[["image_loc", "image_proces"]],
    on="image_loc",
    how="left"
)


In [ ]:
products_image_data.head(3)

___
#### **2.3 Extraction des features**

Nous allons utiliser l'algorithme SIFT (Scale Invariant Feature Transform)

La méthodoligie d'extraction de features par SIFT se fait en 3 étapes :

- Récupérer les descripteurs de chaque image par un algorithme de type SIFT
- Clusteriser l'ensemble de tous les descripteurs
- Construction de l'histogramme pour chaque image (Bag of Visual Words)

____
##### **2.3.1 Extraction des descripteurs sur une image (illustration)**

In [151]:
mug = cv.imread(r"C:\Users\hp\Desktop\Projet_ML\Base\Images\3c4ca34c50a5437a1bcc42b72fc1351f.jpg")

In [ ]:
# Transformation del'image en matrice de pixels pour OpenCV
mug = np.array(mug)

# Extraction des keypoints et descripteurs SIFT
sift_keypoints, sift_descripteurs = pf.gen_sift_features(mug)

print(f"L'image contient {sift_descripteurs.shape[0]} descripteurs SIFT")
print(f'Chaque descripteur est un vecteur de longueur {sift_descripteurs.shape[1]}')

In [ ]:
# Visualisation des points d'intérêt
sift_mug_keypoints = cv.drawKeypoints(mug, sift_keypoints, None)

plt.figure(figsize=(8, 4))
plt.grid(False)

# Image originale
plt.subplot(1, 2, 1)
plt.grid(False)
plt.title('Image originale')
plt.imshow(mug)

# Image avec les points clés
plt.subplot(1, 2, 2)
plt.grid(False)
plt.imshow(sift_mug_keypoints)
plt.title('SIFT Points clés')

plt.show()

___
##### **2.3.3 Etape 1: Extraction des descripteurs SIFT sur l'ensemble des images**

Nous détectons les features, extrayons les descripteurs de chaque image du dataset et construisons un dictionnaire visuel.

In [ ]:
# Chargement des images pré-traitées dans le dictionnaire images
images = pf.load_image_in_dict(r"C:\Users\hp\Desktop\Projet_ML\Base\Images")
print(f'{len(images)} images chargées')

In [ ]:
from PIL import Image
import os

# Chemin vers l'image d'origine
input_path = r"C:\Users\hp\Desktop\Projet_ML\Base\Images\5518124b75d6c6dfee4c2e4c0cfa716a.jpg"

# Chemin vers l'image redimensionnée
output_path = r"C:\Users\hp\Desktop\Projet_ML\Base\Images\resized_5518124b75d6c6dfee4c2e4c0cfa716a.jpg"

# Taille maximale (tu peux ajuster si besoin)
max_size = (1024, 1024)

try:
    # Ouvrir l’image
    img = Image.open(input_path)

    # Redimensionner en gardant les proportions
    img.thumbnail(max_size, Image.LANCZOS)

    # Sauvegarder l’image redimensionnée
    img.save(output_path)

    print(f"[✅] Image redimensionnée enregistrée à : {output_path}")
except FileNotFoundError:
    print(f"[❌] Image introuvable : {input_path}")
except Exception as e:
    print(f"[❌] Erreur lors du redimensionnement : {e}")


In [ ]:
sifts = pf.sift_features_batch(images, batch_size=100)

In [139]:
#Extraction des descripteurs SIFT
#sifts = pf.sift_features(images)

In [ ]:
# Liste des descripteurs
descriptor_list = sifts[0] 
# Liste des vecteurs par images (dictionnaires clé=nom_image, vecteur)
all_bovw_feature = sifts[1]

print("Nombre de descripteurs : ", len(descriptor_list))

___
##### **2.3.4 Etape 2: Clustering des descripteurs**

Ensuite, nous créons des clusters à partir des descripteurs (avec K-Means). Le centre de chaque cluster sera utilisé comme vocabulaire du dictionnaire visuel.

Une règle empirique consiste à créer k clusters avec k = nombre de catégories * 10 (dans notre cas, c'est 70).

In [143]:
N_CLUSTERS = 70

In [ ]:
# Récupère les centres de clusters qui sont les mots visuels    
visual_words = pf.kmeans(N_CLUSTERS, descriptor_list) 

____
##### **2.3.5 Etape 3: Construction de l'histogramme pour chaque image (Bag of Visual Words)** 

Enfin, pour chaque image, on fait un histogramme de fréquence à partir des vocabulaires (les centres des clusters obtenus à l'étape précédente) et de la fréquence des vocabulaires dans l'image. Ces histogrammes sont nos sacs de mots visuels (BOVW).

In [ ]:
# Création des histogrammes  
bovw = pf.image_class(all_bovw_feature, visual_words)

In [ ]:
# Dataframe de sauvegarde des BOVW SIFT
df_bovw = pd.DataFrame({'image': bovw.keys(),'sift_bovw': bovw.values()})
# Ajout des BOVW en np.array d'une liste
df_bovw['Histo'] = [row[0] for row in df_bovw['sift_bovw']]
# Ajout de la catégorie
df_bovw = df_bovw.merge(products_image_data[['image', 'image_loc','category']], how='left', on='image')

df_bovw.head(3)

In [ ]:
# La matrice des histogrammes
hist_vectors=[]
for key,value in bovw.items():
    hist_vectors.append(value[0])
    
im_features = np.asarray(hist_vectors)

In [ ]:
im_features.shape

**Visualisation de l'histogramme pour notre image exemple**

In [ ]:
# Récupération de l'historgramme
df_visu = df_bovw[df_bovw['image'] ==
                       '3c4ca34c50a5437a1bcc42b72fc1351f.jpg']['sift_bovw']
histo_mug = df_visu.to_list()[0][0]
plt.figure(figsize=(20, 7))
plt.title('SIFT Histogramme Image Mug en exemple', fontsize=30)
plt.xlabel("SIFT Visual words", fontsize=24)
plt.ylabel("Fréquence des VW", fontsize=24)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.grid(False)
plt.plot(histo_mug)
plt.show()

<div style="background-color:#d4f0ff; font-size:20px;padding:20px; border-left:25px solid #1c8adb;">
    <strong> 3. REDUCTION DE DIMENSION </strong> 
</div>

Nous allons utiliser 2 techniques :
- Analyse en Composantes Principales (ACP)
- t-distributed stochastic neighbor embedding (t-SNE)

### 3.1 Textes

Les matrices BOW et TF-IDF donnent des matrices creuses et imposantes. 

Une réduction dimensionnelle pourra aider à "condenser" ces matrices en faisant ressortir de nouvelles features qui "résumeront" les features initiales. 

Cela nous permettra par la suite de visualiser ces jeux de données, tout en réduisant les temps de calcul.

#### 3.1.1 PCA

La réduction PCA permet de créer des features décorrélées entre elles, et de diminuer leur dimension, tout en gardant un niveau de variance expliquée élevé (99% dans notre cas)

##### 3.1.1.1 BOW

In [ ]:
print("Dimensions dataset avant réduction PCA : ", bow_vect.shape)
pca = decomposition.PCA(n_components=0.99)
pca_results_bow= pca.fit_transform(bow_vect)
print("Dimensions dataset après réduction PCA : ", pca_results_bow.shape)

##### 3.1.1.2 TF-IDF

In [ ]:
print("Dimensions dataset avant réduction PCA : ", tfidf_vect.shape)
pca = decomposition.PCA(n_components=0.99)
pca_results_tfidf= pca.fit_transform(tfidf_vect)
print("Dimensions dataset après réduction PCA : ", pca_results_tfidf.shape)

##### 3.1.1.3 Word2Vec

In [ ]:
print("Dimensions dataset avant réduction PCA : ", vector_w2v_desc_lem.shape)
pca = decomposition.PCA(n_components=0.99)
pca_results_word2vec= pca.fit_transform(vector_w2v_desc_lem)
print("Dimensions dataset après réduction PCA : ", pca_results_word2vec.shape)


#### 3.1.2 TSNE

Réduction de dimension en 2 composantes T-SNE pour affichage en 2D des textes

##### 3.1.2.1 BOW

In [ ]:
tsne = manifold.TSNE(n_components=2, perplexity=30, 
                     n_iter=2000, init='random', random_state=6)
tsne_results_bow = tsne.fit_transform(pca_results_bow)

df_tsne_bow = pd.DataFrame(tsne_results_bow[:,0:2], columns=['tsne1', 'tsne2'])
df_tsne_bow["class"] = products_text_data["category"]
print(df_tsne_bow.shape)

##### 3.1.2.2 TF-IDF

In [ ]:
tsne = manifold.TSNE(n_components=2, perplexity=30, 
                     n_iter=2000, init='random', random_state=6)
tsne_results_tfidf = tsne.fit_transform(pca_results_tfidf)

df_tsne_tfidf = pd.DataFrame(tsne_results_tfidf[:,0:2], columns=['tsne1', 'tsne2'])
df_tsne_tfidf["class"] = products_text_data["category"]
print(df_tsne_tfidf.shape)

##### 3.1.2.3 Word2Vec

In [ ]:
tsne = manifold.TSNE(n_components=2, perplexity=30, 
                     n_iter=2000, init='random', random_state=6)
tsne_results_word2vec = tsne.fit_transform(pca_results_word2vec)

df_tsne_word2vec = pd.DataFrame(tsne_results_word2vec[:,0:2], columns=['tsne1', 'tsne2'])
df_tsne_word2vec["class"] = products_text_data["category"]
print(df_tsne_word2vec.shape)

#### 3.1.3 Visualisation

Affichage T-SNE selon les catégories d'images

##### 3.1.3.1 BOW

In [ ]:
plt.figure(figsize=(15,10))
sns.scatterplot(
    x="tsne1", y="tsne2", hue="class", data=df_tsne_bow, legend="brief",
    palette=sns.color_palette('tab10', n_colors=7), s=50, alpha=0.8)

plt.title('TSNE (BOW) selon les catégories produit', fontsize = 30, pad = 35, fontweight = 'bold')
plt.xlabel('tsne1', fontsize = 26, fontweight = 'bold')
plt.ylabel('tsne2', fontsize = 26, fontweight = 'bold')
plt.legend(prop={'size': 14}) 

plt.show()

##### 3.1.3.2 TF-IDF

In [ ]:
plt.figure(figsize=(15,10))
sns.scatterplot(
    x="tsne1", y="tsne2", hue="class", data=df_tsne_tfidf, legend="brief",
    palette=sns.color_palette('tab10', n_colors=7), s=50, alpha=0.8)

plt.title('TSNE (TF-IDF) selon les catégories produit', fontsize = 30, pad = 35, fontweight = 'bold')
plt.xlabel('tsne1', fontsize = 26, fontweight = 'bold')
plt.ylabel('tsne2', fontsize = 26, fontweight = 'bold')
plt.legend(prop={'size': 14}) 

plt.show()

##### 3.1.3.3 Word2Vec

In [ ]:
plt.figure(figsize=(15,10))
sns.scatterplot(
    x="tsne1", y="tsne2", hue="class", data=df_tsne_word2vec, legend="brief",
    palette=sns.color_palette('tab10', n_colors=7), s=50, alpha=0.8)

plt.title('TSNE (Word2Vec) selon les catégories produit', fontsize = 30, pad = 35, fontweight = 'bold')
plt.xlabel('tsne1', fontsize = 26, fontweight = 'bold')
plt.ylabel('tsne2', fontsize = 26, fontweight = 'bold')
plt.legend(prop={'size': 14}) 

plt.show()

**Observations**

- Les affichages des données textes (à partir TF-IDF et Word2Vec) permettent de mieux observer une sorte de regroupement des points appartenant à la même catégorie produit. 
- On observe même qu'avec les feaures construits avec TF-IDF, les points appartenant à la catégorie *Watches* sont très bien regroupés et le groupe est bien séparé des autres groupes. 

### 3.2 Images

___
#### **3.1 PCA**

La réduction PCA permet de créer des features décorrélées entre elles, et de diminuer leur dimension, tout en gardant un niveau de variance expliquée élevé (99% dans notre cas)

In [70]:
print("Dimensions dataset avant réduction PCA : ", im_features.shape)
pca = decomposition.PCA(n_components=0.99)
pca_results_img = pca.fit_transform(im_features)
print("Dimensions dataset après réduction PCA : ", pca_results_img.shape)

Dimensions dataset avant réduction PCA :  (1050, 70)
Dimensions dataset après réduction PCA :  (1050, 59)


___
#### **3.2 TSNE**

Réduction de dimension en 2 composantes T-SNE pour affichage en 2D des images

In [ ]:
tsne = manifold.TSNE(n_components=2, perplexity=30, 
                     n_iter=2000, init='random', random_state=6)
tsne_results_img = tsne.fit_transform(pca_results_img)

df_tsne_img = pd.DataFrame(tsne_results_img[:,0:2], columns=['tsne1', 'tsne2'])
df_tsne_img["class"] = products_image_data["category"]
print(df_tsne_img.shape)

___
#### **3.3 Visualisation**

Affichage T-SNE selon les catégories d'images

In [ ]:
plt.figure(figsize=(15,10))
sns.scatterplot(
    x="tsne1", y="tsne2", hue="class", data=df_tsne_img, legend="brief",
    palette=sns.color_palette('tab10', n_colors=7), s=50, alpha=0.8)

plt.title('TSNE(SIFT) selon les catégories produit', fontsize = 30, pad = 35, fontweight = 'bold')
plt.xlabel('tsne1', fontsize = 26, fontweight = 'bold')
plt.ylabel('tsne2', fontsize = 26, fontweight = 'bold')
plt.legend(prop={'size': 14}) 

plt.show()

**Observation**

Le regroupement par catégories produits est moins perceptible que ce qu'on a pu observer sur les données textuelles.

<div style="background-color:#d4f0ff; font-size:20px;padding:20px; border-left:25px solid #1c8adb;">
    <strong> 4. CLUSTERING </strong> 
</div>

In [ ]:
N_CLUSTERS = 7 # On a 7 catégories produit

### 4.1 Textes

La visualisation t-SNE sur les features TF-IDF et Word2Vec ayant permis d'observer un meilleur regroupement des catégories, ce sont ces features que nous conserverons pour le clustering.

#### 4.1.1 TF-IDF

In [ ]:
#Création de clusters à partir du T-SNE 
X = df_tsne_tfidf[["tsne1","tsne2"]]

cls = KMeans(n_clusters=N_CLUSTERS, init='k-means++', random_state=0)
cls.fit(X)

df_tsne_tfidf["cluster"] = cls.labels_

sil = silhouette_score(X,cls.labels_,metric="euclidean")
print("Le score de silhouette est", sil)

In [ ]:
# Création des labels à partir du nom de catégorie produit
le = preprocessing.LabelEncoder()
products_text_data["label"] = le.fit_transform(products_text_data["category"])

In [ ]:
le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print(le_name_mapping)

In [ ]:
list_labels = ["Baby Care", "Beauty and Personal Care", "Computers", "Home Decor & Festive Needs", "Home Furnishing", "Kitchen & Dining", "Watches"]

In [ ]:
#Affichage des images selon clusters et calcul ARI de similarité catégories produit / clusters
plt.figure(figsize=(15,10))
sns.scatterplot(
    x="tsne1", y="tsne2",
    hue="cluster",
    palette=sns.color_palette('tab10', n_colors=7), s=50, alpha=0.8,
    data=df_tsne_tfidf,
    legend="brief")

plt.title('TSNE (TF-IDF) selon les clusters', fontsize = 30, pad = 35, fontweight = 'bold')
plt.xlabel('tsne1', fontsize = 26, fontweight = 'bold')
plt.ylabel('tsne2', fontsize = 26, fontweight = 'bold')
plt.legend(prop={'size': 14}) 

plt.show()

labels = products_text_data["label"]
print("ARI : ", metrics.adjusted_rand_score(labels, cls.labels_))

**Qualité de la catégorisation**

In [ ]:
df_tsne_tfidf.groupby("cluster").count()["class"]

In [ ]:
pf.plot_stacked_bar_clust_vs_cat(df_tsne_tfidf["cluster"],
                              df_tsne_tfidf["class"],
                              df_tsne_tfidf,   
                              palette='tab10',
                              figsize=(7,4),
                              bboxtoanchor=(1,1), ylim=(0,250))

In [ ]:
cat_clust_confmat = pf.plot_conf_matrix_cat_vs_clust(df_tsne_tfidf['class'],
                                                  df_tsne_tfidf['cluster'],
                                                  normalize=False,
                                                  margins_sums=True,
                                                  margins_score=True)
with pd.option_context('display.float_format', '{:.0f}'.format):
    display(cat_clust_confmat)

In [ ]:
# Affiche la matrice de confusion (Vraies catégories vs. Clusters)

cm = pf.plot_conf_matrix_cat_vs_clust(df_tsne_tfidf['class'],
                            df_tsne_tfidf['cluster'],
                            normalize=False,
                            margins_sums=False,
                            margins_score=False)
pf.plot_heatmap(cm, "Confusion matrix | true categories vs. clusters",
             figsize=(8, 4), vmin=0, vmax=150, center=75,
                 palette=sns.color_palette("viridis", 20), shape='rect',
                 fmt='.0f', robust=False, fig=None, ax=None)

#### 4.1.2 Word2Vec

In [ ]:
#Création de clusters à partir du T-SNE 
X = df_tsne_word2vec[["tsne1","tsne2"]]

cls = KMeans(n_clusters=N_CLUSTERS, init='k-means++', random_state=0)
cls.fit(X)

df_tsne_word2vec["cluster"] = cls.labels_

sil = silhouette_score(X,cls.labels_,metric="euclidean")
print("Le score de silhouette est", sil)

In [ ]:
#Affichage des images selon clusters et calcul ARI de similarité catégories images / clusters
plt.figure(figsize=(15,10))
sns.scatterplot(
    x="tsne1", y="tsne2",
    hue="cluster",
    palette=sns.color_palette('tab10', n_colors=7), s=50, alpha=0.8,
    data=df_tsne_word2vec,
    legend="brief")

plt.title('TSNE (Word2Vec) selon les clusters', fontsize = 30, pad = 35, fontweight = 'bold')
plt.xlabel('tsne1', fontsize = 26, fontweight = 'bold')
plt.ylabel('tsne2', fontsize = 26, fontweight = 'bold')
plt.legend(prop={'size': 14}) 

plt.show()

labels = products_text_data["label"]
print("ARI : ", metrics.adjusted_rand_score(labels, cls.labels_))

**Qualité de la catégorisation**

In [ ]:
df_tsne_word2vec.groupby("cluster").count()["class"]

In [ ]:
pf.plot_stacked_bar_clust_vs_cat(df_tsne_word2vec["cluster"],
                              df_tsne_word2vec["class"],
                              df_tsne_word2vec,   
                              palette='tab10',
                              figsize=(7,4),
                              bboxtoanchor=(1,1), ylim=(0,300))

In [ ]:
cat_clust_confmat = pf.plot_conf_matrix_cat_vs_clust(df_tsne_word2vec['class'],
                                                  df_tsne_word2vec['cluster'],
                                                  normalize=False,
                                                  margins_sums=True,
                                                  margins_score=True)
with pd.option_context('display.float_format', '{:.0f}'.format):
    display(cat_clust_confmat)

In [ ]:
# Affiche la matrice de confusion (Vraies catégories vs. Clusters)

cm = pf.plot_conf_matrix_cat_vs_clust(df_tsne_word2vec['class'],
                            df_tsne_word2vec['cluster'],
                            normalize=False,
                            margins_sums=False,
                            margins_score=False)
pf.plot_heatmap(cm, "Confusion matrix | true categories vs. clusters",
             figsize=(8, 4), vmin=0, vmax=150, center=75,
                 palette=sns.color_palette("viridis", 20), shape='rect',
                 fmt='.0f', robust=False, fig=None, ax=None)

**Recap**

*TF-IDF*

- ARI = 0.59
- Silhouette = 0.48
- 220 erreurs de catégorisation

*Word2Vec*

- ARI = 0.34
- Silhouette = 0.45
- 485 erreurs de catégorisation

**Observation générale**
- les scores ARI et de silhouette sont plutôt moyennement bons sur les features textuels.
- ce qui suggère qu'une classification des produits à partir des données textuelles serait plutôt faisable

### 4.2 Images

In [ ]:
#Création de clusters à partir du T-SNE 
X = df_tsne_img[["tsne1","tsne2"]]

cls = KMeans(n_clusters=N_CLUSTERS, init='k-means++', random_state=0)
cls.fit(X)

df_tsne_img["cluster"] = cls.labels_

sil = silhouette_score(X,cls.labels_,metric="euclidean")
print("Le score de silhouette est", sil)

In [ ]:
# Création des labels à partir du nom de catégory produit
products_image_data["label"] = le.fit_transform(products_image_data["category"])

In [ ]:
le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print(le_name_mapping)

In [ ]:
#Affichage des images selon clusters et calcul ARI de similarité catégories images / clusters
plt.figure(figsize=(15,10))
sns.scatterplot(
    x="tsne1", y="tsne2",
    hue="cluster",
    palette=sns.color_palette('tab10', n_colors=7), s=50, alpha=0.8,
    data=df_tsne_img,
    legend="brief")

plt.title('TSNE (SIFT) selon les clusters', fontsize = 30, pad = 35, fontweight = 'bold')
plt.xlabel('tsne1', fontsize = 26, fontweight = 'bold')
plt.ylabel('tsne2', fontsize = 26, fontweight = 'bold')
plt.legend(prop={'size': 14}) 

plt.show()

labels = products_image_data["label"]
print("ARI : ", metrics.adjusted_rand_score(labels, cls.labels_))

**Observations** 
- le score ARI est très mauvais
- le score de silhouette est moins bon que celui obtenu sur les données textes

**Qualité de la catégorisation**

In [ ]:
df_tsne_img.groupby("cluster").count()["class"]

In [ ]:
pf.plot_stacked_bar_clust_vs_cat(df_tsne_img["cluster"],
                              df_tsne_img["class"],
                              df_tsne_img,   
                              palette='tab10',
                              figsize=(7,4),
                              bboxtoanchor=(1,1), ylim=(0,250))

In [95]:
cat_clust_confmat = pf.plot_conf_matrix_cat_vs_clust(df_tsne_img['class'],
                                                  df_tsne_img['cluster'],
                                                  normalize=False,
                                                  margins_sums=True,
                                                  margins_score=True)
with pd.option_context('display.float_format', '{:.0f}'.format):
    display(cat_clust_confmat)

cluster,6,0,1,3,4,5,2,cat_sum,cat_matching_score_pct
class,,,,,,,,,
Baby Care,24,16,27,24,22,21,16,150,16
Beauty and Personal Care,16,26,17,29,30,17,15,150,17
Computers,15,25,23,21,30,23,13,150,15
Home Decor & Festive Needs,16,23,25,33,22,17,14,150,22
Home Furnishing,13,19,20,26,30,23,19,150,20
Kitchen & Dining,18,19,19,21,31,26,16,150,17
Watches,18,17,20,21,31,23,20,150,13
clust_sum,120,145,151,175,196,150,113,-,-
clust_matching_score_pct,20,18,15,19,15,17,18,-,-


In [ ]:
# Affiche la matrice de confusion (Vraies catégories vs. Clusters)

cm = pf.plot_conf_matrix_cat_vs_clust(df_tsne_img['class'],
                            df_tsne_img['cluster'],
                            normalize=False,
                            margins_sums=False,
                            margins_score=False)
pf.plot_heatmap(cm, "Confusion matrix | true categories vs. clusters",
             figsize=(8, 4), vmin=0, vmax=150, center=75,
                 palette=sns.color_palette("viridis", 20), shape='rect',
                 fmt='.0f', robust=False, fig=None, ax=None)

**Observations**

- ARI = -0.00019
- Silhouette = 0.37
- 867 erreurs de catégorisation

Les catégorisation avec les features SIFT ne permet pas vraiment de retrouver les vraies catégories produit (score ARI très mauvais)

___
## 5. CONCLUSION

- L'utilisation des données texte (features TF-IDF) permet une meilleur catégorisation des produits par rapport aux données image.
- La classification avec les données images pouraient certainement être améliorée en utilisant un algorithme de type CNN (Transfer Learning). Nous allons le tester dans la section ANNEXE

>**Nous validons la faisabilité de la mise en oeuvre d'un moteur de classification automatique des produits**




___
## 6. ANNEXE 

### 6.1  EXPLORATION DES TOPICS

On va tester 2 approches non supervisées, afin de voir si les topics abordés dans les documents permettraient de classifier un produit : 
- Latent Dirichlet Allocation (LDA)
- Negative Matrix Factorization (NMF)

#### 6.1.1 Latent Dirichlet Allocation (LDA)

In [97]:
n_topics = 7
n_top_words = 10

# Créer le modèle LDA
lda = LatentDirichletAllocation(n_components=n_topics, 
                                max_iter=5, 
                                learning_method='online', 
                                learning_offset=50,
                                random_state=0)

##### 6.1.1.1 Données Bag of Words

In [ ]:
# Fitter sur les données
lda.fit(bow_vect)

In [ ]:
print("Catégories in LDA model (BOW):")
bow_feature_names = bow.get_feature_names()

pf.print_top_words(lda, bow_feature_names, n_top_words)

Analysons les proportions de produits des différentes catégories connues pour chaque topic, afin de voir s'il est possible de faire des couples (catégorie, topic) uniques.

In [100]:
pf.get_categories_per_topic(n_topics, 
                            lda, 
                            bow_vect, 
                            products_text_data)

,% Home Furnishing,% Baby Care,% Watches,% Home Decor & Festive Needs,% Kitchen & Dining,% Beauty and Personal Care,% Computers
Topic,,,,,,,
0.0,1.111111,52.222222,7.222222,3.888889,5.555556,13.888889,16.111111
1.0,25.000000,6.250000,0.000000,25.000000,31.250000,6.250000,6.250000
2.0,0.000000,0.000000,0.000000,0.000000,0.000000,85.714286,14.285714
3.0,0.000000,7.692308,23.076923,0.000000,15.384615,17.948718,35.897436
4.0,32.773109,7.983193,0.000000,24.369748,29.411765,3.361345,2.100840
5.0,12.406015,5.827068,24.060150,14.661654,8.834586,18.796992,15.413534
6.0,0.000000,5.263158,0.000000,7.894737,31.578947,7.894737,47.368421


On pourrait associer certain topic à certaines catégories. Par exemple:

- topic #0 : Baby Care (52%)     
- topic #2 : Beauty and Personal Care (86%) 
- topic #6 : Computers (47%)

##### 6.1.1.2 Données TF-IDF

In [ ]:
# Fitter sur les données
lda.fit(tfidf_vect)

In [ ]:
print("Catégories in LDA model (TF-IDF):")
tf_feature_names = tfidf.get_feature_names()

pf.print_top_words(lda, tf_feature_names, n_top_words)

Analysons les proportions de produits des différentes catégories connues pour chaque topic, afin de voir s'il est possible de faire des couples (catégorie, topic) uniques.

In [103]:
pf.get_categories_per_topic(n_topics, 
                            lda, 
                            tfidf_vect, 
                            products_text_data)

,% Home Furnishing,% Baby Care,% Watches,% Home Decor & Festive Needs,% Kitchen & Dining,% Beauty and Personal Care,% Computers
Topic,,,,,,,
0.0,1.680672,7.563025,60.924370,2.941176,8.403361,14.705882,3.781513
1.0,0.000000,0.000000,0.000000,50.000000,0.000000,50.000000,0.000000
2.0,0.000000,0.000000,0.000000,4.347826,86.956522,4.347826,4.347826
3.0,16.300940,4.702194,0.000000,12.852665,12.852665,22.570533,30.721003
4.0,22.488038,27.990431,1.196172,22.966507,8.373206,7.894737,9.090909
5.0,0.000000,0.000000,0.000000,30.000000,10.000000,20.000000,40.000000
6.0,0.000000,0.000000,0.000000,0.000000,86.842105,13.157895,0.000000


On pourrait associer certain topic à certaines catégories. Par exemple:

- topic #0 : Watches (61%)
- topic #2 : Kitchen & Dining (87%)  
- topic #5 : Computers (40%) 
- topic #6 : Kitchen & Dining (87%)

#### 6.1.2 Negative Matrix Factorization (NMF)

In [104]:
n_topics = 7
n_top_words = 10

nmf = NMF(n_components=n_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd')

##### 6.1.2.1 Données Bag of Words

In [ ]:
bow_feature_names = bow.get_feature_names()

nmf.fit(bow_vect)

pf.print_top_words(nmf, bow_feature_names, n_top_words)

Analysons les proportions de produits des différentes catégories connues pour chaque topic, afin de voir s'il est possible de faire des couples (catégorie, topic) uniques.

In [106]:
pf.get_categories_per_topic(n_topics, 
                            nmf, 
                            bow_vect,
                            products_text_data)

,% Home Furnishing,% Baby Care,% Watches,% Home Decor & Festive Needs,% Kitchen & Dining,% Beauty and Personal Care,% Computers
Topic,,,,,,,
0.0,29.302326,9.302326,0.465116,31.162791,15.813953,8.837209,5.116279
1.0,0.000000,1.388889,0.000000,0.000000,98.611111,0.000000,0.000000
2.0,17.452830,8.254717,0.000000,18.396226,9.198113,24.528302,22.169811
3.0,0.000000,0.000000,0.000000,3.030303,6.060606,9.090909,81.818182
4.0,9.615385,86.538462,0.000000,0.000000,0.000000,3.846154,0.000000
5.0,6.000000,8.000000,0.000000,8.000000,8.000000,36.000000,34.000000
6.0,0.000000,0.000000,98.026316,0.000000,0.000000,1.315789,0.657895


On pourrait associer certain topic à certaines catégories. Par exemple:

- topic #1 : Kitchen & Dining (99%)     
- topic #3 : Computers (82%)      
- topic #4 : Baby Care (87%)
- topic #6 : Watches (98%)

##### 6.1.2.2 Données TF-IDF

In [ ]:
tf_feature_names = tfidf.get_feature_names()

nmf.fit(tfidf_vect)

pf.print_top_words(nmf, tf_feature_names, n_top_words)

Analysons les proportions de produits des différentes catégories connues pour chaque topic, afin de voir s'il est possible de faire des couples (catégorie, topic) uniques.

In [108]:
pf.get_categories_per_topic(n_topics, 
                            nmf, 
                            tfidf_vect,
                            products_text_data)

,% Home Furnishing,% Baby Care,% Watches,% Home Decor & Festive Needs,% Kitchen & Dining,% Beauty and Personal Care,% Computers
Topic,,,,,,,
0.0,0.000000,1.104972,82.320442,3.867403,4.972376,4.972376,2.762431
1.0,0.000000,3.076923,0.000000,10.769231,86.153846,0.000000,0.000000
2.0,23.762376,51.980198,0.000000,11.881188,5.940594,5.940594,0.495050
3.0,5.586592,7.821229,0.000000,22.905028,11.452514,31.843575,20.391061
4.0,0.000000,0.000000,0.000000,25.581395,69.767442,4.651163,0.000000
5.0,81.000000,9.000000,0.000000,9.000000,0.000000,1.000000,0.000000
6.0,0.990099,3.960396,0.990099,9.900990,1.980198,11.881188,70.297030


On pourrait associer certain topic à certaines catégories. Par exemple:

- topic 0 : Watches (82%)            
- topic 1 : Kitchen & Dining (86%)           
- topic 2 : Baby Care (52%)
- topic 4 : Kitchen & Dining (70%)
- topic 5 : Home Furnishing (81%)               
- topic 6 : Computers (70%)

### 6.2  TRANSFER LEARNING

Nous allons implémenter un algorithme de type CNN Transfer Learning en utilisant le modèle VGG-16

- VGG-16 est une version du réseau de neurones convolutif VGG-Net.
- VGG-16 est constitué de plusieurs couches, dont 13 couches de convolution et 3 fully-connected. Il doit donc apprendre les poids de 16 couches.
- Il prend en entrée une image en couleurs de taille 224  × 224 px et la classifie dans une des 1000 classes. Il renvoie donc un vecteur de taille 1000, qui contient les probabilités d'appartenance à chacune des classes. 

- le Transfer Learning consiste à utiliser le modèle VGG16 pré-entraîné sur ImageNet (ImageNet est un projet de recherche visant à développer une grande base de données d'images avec des annotations, c'est-à-dire des images et leurs descriptions).

**3 étapes**

>1. ÉTAPE 1 - Chargement des images, extraction des features (Preprocess each image according to the input accepted by the transfer learning model and convert each image to the respective vector by using the weights from the transfer learning model. Then flatten and store all the image weights in a list)

>2. ÉTAPE 2 - Réduction de dimension (Apply TSNE dimension reduction (initialized  with a PCA))

>3. ÉTAPE 3 - Clustering (Apply clustering with k-means)

#### 6.2.1 ÉTAPE 1 - Chargement des images, extraction des features

In [ ]:
# Utilisation de VGG16 pré-entraîné sur ImageNet avec les images d'origine
# VGG16 imports
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input

# Liste
vgg16_all_features = []

# Instanciation du modèle
model_vgg16 = VGG16(weights='imagenet', include_top=False)
# Résumé de l'architecture du modèle
model_vgg16.summary()

for rep_image in products_image_data['image_loc']:

    # Charger l'image et la redimensionner à la taille
    # requise de 224×224 pixels.
    img = image.load_img(rep_image, target_size=(224, 224))
    # Convertir les pixels en un tableau NumPy afin de pouvoir travailler
    # avec dans Keras
    img = image.img_to_array(img)
    # Redimensionnement
    img = np.expand_dims(img, axis=0)
    # Préparer de nouvelles entrées pour le réseau.
    img = preprocess_input(img)

    # obtenir une prédiction de la probabilité d'appartenance
    # de l'image à chacun des 1000 types d'objets connus.
    vgg16_feature = model_vgg16.predict(img)
    # Ajouter la feature prédite en nparray à la liste
    vgg16_all_features.append(np.array(vgg16_feature).flatten())

vgg16_all_features = np.array(vgg16_all_features)

In [110]:
# Dataframe de sauvegarde des BOVW VGG16
# Association des features à chaque image
products_image_data['vgg16_bovw'] = [vgg16_all_features[i] for i in range(products_image_data.shape[0])]
# Dataframe de travail
df_vgg16_bovw = products_image_data[['image', 'image_loc', 'category', 'vgg16_bovw']]
# Constituer le dataframe de vecteurs de BOVW VGG16
df_vgg16_vectors = pd.DataFrame.from_records(vgg16_all_features)

#### 6.2.2 ÉTAPE 2 - Réduction de dimension

**PCA**

La réduction PCA tout en gardant 99% de variance expliquée 

In [ ]:
print("Dimensions dataset avant réduction PCA : ", df_vgg16_vectors.shape)
pca = decomposition.PCA(n_components=0.99)
pca_results_vgg16 = pca.fit_transform(df_vgg16_vectors)
print("Dimensions dataset après réduction PCA : ", pca_results_vgg16.shape)

**TSNE**

Réduction de dimension en 2 composantes T-SNE pour affichage en 2D des images

In [ ]:
tsne = manifold.TSNE(n_components=2, perplexity=30, 
                     n_iter=2000, init='random', random_state=6)
tsne_results_vgg16 = tsne.fit_transform(pca_results_vgg16)

df_tsne_vgg16 = pd.DataFrame(tsne_results_vgg16[:,0:2], columns=['tsne1', 'tsne2'])
df_tsne_vgg16["class"] = products_image_data["category"]
print(df_tsne_vgg16.shape)

**Visualisation**

Affichage T-SNE selon les catégories d'images

In [ ]:
plt.figure(figsize=(15,10))
sns.scatterplot(
    x="tsne1", y="tsne2", hue="class", data=df_tsne_vgg16, legend="brief",
    palette=sns.color_palette('tab10', n_colors=7), s=50, alpha=0.8)

plt.title('TSNE (VGG-16) selon les catégories produit', fontsize = 30, pad = 35, fontweight = 'bold')
plt.xlabel('tsne1', fontsize = 26, fontweight = 'bold')
plt.ylabel('tsne2', fontsize = 26, fontweight = 'bold')
plt.legend(prop={'size': 14}) 

plt.show()

**Observation**

Le regroupement par catégories produits semble meilleur que ce qu'on a pu observer sur les données SIFT.

#### 6.2.3 ÉTAPE 3 - Clustering

In [ ]:
#Création de clusters à partir du T-SNE 
X = df_tsne_vgg16[["tsne1","tsne2"]]

cls = KMeans(n_clusters=N_CLUSTERS, init='k-means++', random_state=0)
cls.fit(X)

df_tsne_vgg16["cluster"] = cls.labels_

sil = silhouette_score(X,cls.labels_,metric="euclidean")
print("Le score de silhouette est", sil)

In [ ]:
#Affichage des images selon clusters et calcul ARI de similarité catégories images / clusters
plt.figure(figsize=(15,10))
sns.scatterplot(
    x="tsne1", y="tsne2",
    hue="cluster",
    palette=sns.color_palette('tab10', n_colors=7), s=50, alpha=0.8,
    data=df_tsne_vgg16,
    legend="brief")

plt.title('TSNE (VGG-16) selon les clusters', fontsize = 30, pad = 35, fontweight = 'bold')
plt.xlabel('tsne1', fontsize = 26, fontweight = 'bold')
plt.ylabel('tsne2', fontsize = 26, fontweight = 'bold')
plt.legend(prop={'size': 14}) 

plt.show()

labels = products_image_data["label"]
print("ARI : ", metrics.adjusted_rand_score(labels, cls.labels_))

**Observations** 
- le score ARI amélioré (par rapport à celui obtenu sur avec les features SIFT)
- le score de silhouette est egalement amélioré mais reste moins bon que celui obtenu sur les données textes

**Qualité de la catégorisation**

In [ ]:
df_tsne_vgg16.groupby("cluster").count()["class"]

In [ ]:
pf.plot_stacked_bar_clust_vs_cat(df_tsne_vgg16["cluster"],
                              df_tsne_vgg16["class"],
                              df_tsne_vgg16,   
                              palette='tab10',
                              figsize=(7,4),
                              bboxtoanchor=(1,1), ylim=(0,290))

In [118]:
cat_clust_confmat = pf.plot_conf_matrix_cat_vs_clust(df_tsne_vgg16['class'],
                                                  df_tsne_vgg16['cluster'],
                                                  normalize=False,
                                                  margins_sums=True,
                                                  margins_score=True)
with pd.option_context('display.float_format', '{:.0f}'.format):
    display(cat_clust_confmat)

cluster,5,4,6,2,0,1,3,cat_sum,cat_matching_score_pct
class,,,,,,,,,
Baby Care,87,0,5,26,30,2,0,150,58
Beauty and Personal Care,13,84,9,34,4,6,0,150,56
Computers,7,3,50,80,7,3,0,150,33
Home Decor & Festive Needs,4,9,14,59,62,0,2,150,39
Home Furnishing,55,6,3,32,47,7,0,150,31
Kitchen & Dining,0,4,34,30,2,80,0,150,53
Watches,1,0,9,7,4,0,129,150,86
clust_sum,167,106,124,268,156,98,131,-,-
clust_matching_score_pct,52,79,40,22,30,82,98,-,-


In [ ]:
# Affiche la matrice de confusion (Vraies catégories vs. Clusters)

cm = pf.plot_conf_matrix_cat_vs_clust(df_tsne_vgg16['class'],
                            df_tsne_vgg16['cluster'],
                            normalize=False,
                            margins_sums=False,
                            margins_score=False)
pf.plot_heatmap(cm, "Confusion matrix | true categories vs. clusters",
             figsize=(8, 4), vmin=0, vmax=150, center=75,
                 palette=sns.color_palette("viridis", 20), shape='rect',
                 fmt='.0f', robust=False, fig=None, ax=None)

**Observations**

- ARI = 0.28
- Silhouette = 0.38
- 514 erreurs de catégorisation
- On a une meilleure classification des images en utilisation les features obtenus par Tranfer Learning

In [ ]:
# src/features/build_features.py
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input
import numpy as np

def extract_image_features(img_paths):
    model = ResNet50(weights='imagenet', include_top=False, pooling='avg')
    features = []
    
    for path in img_paths:
        if path is None:
            features.append(np.zeros(2048))  # Dimension des features ResNet50
            continue
            
        img = image.load_img(path, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        features.append(model.predict(x).flatten())
    
    return np.array(features)

def build_complete_features(df, text_vectorizer, image_features):
    # Features textuelles
    text_features = text_vectorizer.transform(df['description'])
    
    # Combinaison
    combined_features = np.hstack([text_features.toarray(), image_features])
    
    return combined_features